# Path

In [51]:
import sys
import os

project_path = r'/home/craxiss/Documents/projects/kapsul_etkinlik_v2'

os.chdir(project_path)
sys.path.append(project_path)

# Import

In [52]:
from helpers import column_formatter, first_letter_id_generator
from charset_normalizer import detect
from config import data_path as path
from string import ascii_lowercase
from json import load, dump
import pandas as pd
import openpyxl

# Read

In [53]:
information = pd.read_excel(path+'/3-result/information.xlsx', index_col='Unnamed: 0')

with pd.ExcelFile(path + '/2-cleaned/basvuru.xlsx') as appeals_xlsx:
    appeals = pd.read_excel(appeals_xlsx, index_col='Unnamed: 0', sheet_name='Başvurular')
    lessons_all = pd.read_excel(appeals_xlsx, index_col='Unnamed: 0', sheet_name='Dersler')


with pd.ExcelFile(path + '/2-cleaned/yoklama.xlsx') as rolls_xlsx:
    rolls_info = pd.read_excel(rolls_xlsx, 'General Info', index_col='Unnamed: 0')

    rolls = dict()
    for sheet in rolls_info.index:
        rolls[sheet] = pd.read_excel(rolls_xlsx, sheet)

information['id'] = information.index

# Concatting

In [54]:
def name_indexer(df):
    df = df.copy()
    df.index = df.name
    return df.T

information = name_indexer(information)

## Appeals

In [55]:
events_dict = dict()
lessons_kw = lessons_all.to_dict()[0]
for row in appeals.iterrows():

    row = dict(row[1])
    name = row['name']
    person = dict(information[name])

    if type(row['lessons']) != str:
        lessons = []

    else:
        lessons = row['lessons'].split(',')

    lessons = list(map(lambda x: lessons_kw[x.strip()], lessons))

    if person['id'] not in events_dict:
        events_dict[person['id']] = {'name': name,
                                     'appeals': lessons, 
                                     'rolls': list()}
    else:
        events_dict[person['id']]['appeals'].extend(lessons)


## Rolls

In [57]:
for sheet in rolls:
    for name in rolls[sheet].name:
        id = information[name][4]
        if id not in events_dict:
            events_dict[id] = {'name': name,
                               'appeals': list(),
                               'rolls': list()}

        events_dict[id]['rolls'].append(sheet)

# Write

In [58]:
events_per_person = pd.DataFrame(events_dict).T
for column in ['appeals', 'rolls']:
    events_per_person[column] = events_per_person[column].str.join(',')

In [59]:
with pd.ExcelWriter(path+'3-result/events_per_person.xlsx', mode='w') as writer:
    events_per_person.to_excel(writer, sheet_name='Events')